fastNLP上手教程
-------

fastNLP提供方便的数据预处理，训练和测试模型的功能

In [3]:
import sys
sys.path.append('/Users/yh/Desktop/fastNLP/fastNLP/')

DataSet & Instance
------

fastNLP用DataSet和Instance保存和处理数据。每个DataSet表示一个数据集，每个Instance表示一个数据样本。一个DataSet存有多个Instance，每个Instance可以自定义存哪些内容。

有一些read_*方法，可以轻松从文件读取数据，存成DataSet。

In [4]:
from fastNLP import DataSet
from fastNLP import Instance

# 从csv读取数据到DataSet
dataset = DataSet.read_csv('../sentence.csv', headers=('raw_sentence', 'label'), sep='\t')
print(len(dataset))

8529


In [5]:
# 使用数字索引[k]，获取第k个样本
print(dataset[0])

# 索引也可以是负数
print(dataset[-3])

{'raw_sentence': A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .,
'label': 1}
{'raw_sentence': -LRB- Tries -RRB- to parody a genre that 's already a joke in the United States .,
'label': 1}


## Instance
Instance表示一个样本，由一个或多个field（域，属性，特征）组成，每个field有名字和值。

在初始化Instance时即可定义它包含的域，使用 "field_name=field_value"的写法。

In [6]:
# DataSet.append(Instance)加入新数据
dataset.append(Instance(raw_sentence='fake data', label='0'))
dataset[-1]

{'raw_sentence': fake data,
'label': 0}

## DataSet.apply方法
数据预处理利器

In [7]:
# 将所有数字转为小写
dataset.apply(lambda x: x['raw_sentence'].lower(), new_field_name='raw_sentence')
print(dataset[0])

{'raw_sentence': a series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .,
'label': 1}


In [8]:
# label转int
dataset.apply(lambda x: int(x['label']), new_field_name='label')
print(dataset[0])

{'raw_sentence': a series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .,
'label': 1}


In [9]:
# 使用空格分割句子
def split_sent(ins):
    return ins['raw_sentence'].split()
dataset.apply(split_sent, new_field_name='words')
print(dataset[0])

RuntimeError: Cannot create FieldArray with an empty list.

In [17]:
# 增加长度信息
dataset.apply(lambda x: len(x['words']), new_field_name='seq_len')
print(dataset[0])

{'raw_sentence': a series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .,
'label': 1,
'words': ['a', 'series', 'of', 'escapades', 'demonstrating', 'the', 'adage', 'that', 'what', 'is', 'good', 'for', 'the', 'goose', 'is', 'also', 'good', 'for', 'the', 'gander', ',', 'some', 'of', 'which', 'occasionally', 'amuses', 'but', 'none', 'of', 'which', 'amounts', 'to', 'much', 'of', 'a', 'story', '.'],
'seq_len': 37}


## DataSet.drop
筛选数据

In [19]:
dataset.drop(lambda x: x['seq_len'] <= 3)
print(len(dataset))

38


## 配置DataSet
1. 哪些域是特征，哪些域是标签
2. 切分训练集/验证集

In [20]:
# 设置DataSet中，哪些field要转为tensor

# set target，loss或evaluate中的golden，计算loss，模型评估时使用
dataset.set_target("label")
# set input，模型forward时使用
dataset.set_input("words")

In [21]:
# 分出测试集、训练集

test_data, train_data = dataset.split(0.3)
print(len(test_data))
print(len(train_data))

27
11

Vocabulary
------

fastNLP中的Vocabulary轻松构建词表，将词转成数字

In [22]:
from fastNLP import Vocabulary

# 构建词表, Vocabulary.add(word)
vocab = Vocabulary(min_freq=2)
train_data.apply(lambda x: [vocab.add(word) for word in x['words']])
vocab.build_vocab()

# index句子, Vocabulary.to_index(word)
train_data.apply(lambda x: [vocab.to_index(word) for word in x['words']], new_field_name='words')
test_data.apply(lambda x: [vocab.to_index(word) for word in x['words']], new_field_name='words')


print(test_data[0])

{'raw_sentence': that the chuck norris `` grenade gag '' occurs about 7 times during windtalkers is a good indication of how serious-minded the film is .,
'label': 2,
'words': [6, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 10, 8, 24, 1, 5, 1, 1, 2, 15, 10, 3],
'seq_len': 25}


# Model
定义一个PyTorch模型

In [23]:
from fastNLP.models import CNNText
model = CNNText(embed_num=len(vocab), embed_dim=50, num_classes=5, padding=2, dropout=0.1)
model

CNNText(
  (embed): Embedding(
    (embed): Embedding(32, 50, padding_idx=0)
    (dropout): Dropout(p=0.0)
  )
  (conv_pool): ConvMaxpool(
    (convs): ModuleList(
      (0): Conv1d(50, 3, kernel_size=(3,), stride=(1,), padding=(2,))
      (1): Conv1d(50, 4, kernel_size=(4,), stride=(1,), padding=(2,))
      (2): Conv1d(50, 5, kernel_size=(5,), stride=(1,), padding=(2,))
    )
  )
  (dropout): Dropout(p=0.1)
  (fc): Linear(
    (linear): Linear(in_features=12, out_features=5, bias=True)
  )
)

这是上述模型的forward方法。如果你不知道什么是forward方法，请参考我们的PyTorch教程。

注意两点：
1. forward参数名字叫**word_seq**，请记住。
2. forward的返回值是一个**dict**，其中有个key的名字叫**output**。

```Python
    def forward(self, word_seq):
        """

        :param word_seq: torch.LongTensor, [batch_size, seq_len]
        :return output: dict of torch.LongTensor, [batch_size, num_classes]
        """
        x = self.embed(word_seq)  # [N,L] -> [N,L,C]
        x = self.conv_pool(x)  # [N,L,C] -> [N,C]
        x = self.dropout(x)
        x = self.fc(x)  # [N,C] -> [N, N_class]
        return {'output': x}
```

这是上述模型的predict方法，是用来直接输出该任务的预测结果，与forward目的不同。

注意两点：
1. predict参数名也叫**word_seq**。
2. predict的返回值是也一个**dict**，其中有个key的名字叫**predict**。

```
    def predict(self, word_seq):
        """

        :param word_seq: torch.LongTensor, [batch_size, seq_len]
        :return predict: dict of torch.LongTensor, [batch_size, seq_len]
        """
        output = self(word_seq)
        _, predict = output['output'].max(dim=1)
        return {'predict': predict}
```

Trainer & Tester
------

使用fastNLP的Trainer训练模型

In [25]:
from fastNLP import Trainer
from copy import deepcopy
from fastNLP.core.losses import CrossEntropyLoss
from fastNLP.core.metrics import AccuracyMetric


# 更改DataSet中对应field的名称，与模型的forward的参数名一致
# 因为forward的参数叫word_seq, 所以要把原本叫words的field改名为word_seq
# 这里的演示是让你了解这种**命名规则**
train_data.rename_field('words', 'word_seq')
test_data.rename_field('words', 'word_seq')

# 顺便把label换名为label_seq
train_data.rename_field('label', 'label_seq')
test_data.rename_field('label', 'label_seq')

### loss
训练模型需要提供一个损失函数

下面提供了一个在分类问题中常用的交叉熵损失。注意它的**初始化参数**。

pred参数对应的是模型的forward返回的dict的一个key的名字，这里是"output"。

target参数对应的是dataset作为标签的field的名字，这里是"label_seq"。

In [26]:
loss = CrossEntropyLoss(pred="output", target="label_seq")

### Metric
定义评价指标

这里使用准确率。参数的“命名规则”跟上面类似。

pred参数对应的是模型的predict方法返回的dict的一个key的名字，这里是"predict"。

target参数对应的是dataset作为标签的field的名字，这里是"label_seq"。

In [27]:
metric = AccuracyMetric(pred="predict", target="label_seq")

In [30]:
# 实例化Trainer，传入模型和数据，进行训练
# 先在test_data拟合
copy_model = deepcopy(model)
overfit_trainer = Trainer(model=copy_model, train_data=test_data, dev_data=test_data,
                          losser=loss,
                          metrics=metric,
                          save_path=None,
                          batch_size=32,
                          n_epochs=5)
overfit_trainer.train()

training epochs started 2018-12-04 22:51:24


Epoch 1/5. Step:1/5. AccuracyMetric: acc=0.296296


Epoch 2/5. Step:2/5. AccuracyMetric: acc=0.407407


Epoch 3/5. Step:3/5. AccuracyMetric: acc=0.518519


Epoch 4/5. Step:4/5. AccuracyMetric: acc=0.481481


Epoch 5/5. Step:5/5. AccuracyMetric: acc=0.592593


In [31]:
# 用train_data训练，在test_data验证
trainer = Trainer(model=model, train_data=train_data, dev_data=test_data,
                  losser=CrossEntropyLoss(pred="output", target="label_seq"),
                  metrics=AccuracyMetric(pred="predict", target="label_seq"),
                  save_path=None,
                  batch_size=32,
                  n_epochs=5)
trainer.train()
print('Train finished!')

training epochs started 2018-12-04 22:52:01


Epoch 1/5. Step:1/5. AccuracyMetric: acc=0.296296


Epoch 2/5. Step:2/5. AccuracyMetric: acc=0.222222


Epoch 3/5. Step:3/5. AccuracyMetric: acc=0.259259


Epoch 4/5. Step:4/5. AccuracyMetric: acc=0.296296


Epoch 5/5. Step:5/5. AccuracyMetric: acc=0.259259


Train finished!


In [33]:
# 调用Tester在test_data上评价效果
from fastNLP import Tester

tester = Tester(data=test_data, model=model, metrics=AccuracyMetric(pred="predict", target="label_seq"),
                batch_size=4)
acc = tester.test()
print(acc)

[tester] 
AccuracyMetric: acc=0.259259
{'AccuracyMetric': {'acc': 0.259259}}
